# 実現方法調査

[Qiita: Kaggle Image Matching Challenge 2022 まとめ](https://qiita.com/fam_taro/items/bdc47e944d4612a0eac1)  
上の記事では基礎行列をDeep Learningで推定する方法を紹介している。  
具体的には、LoFTR(Detector - Free Local Feature Matching with Transformers)を用いて、  
２枚の画像から対応点を取得しcv2.findEssentialMat()で基礎行列を推定する。  
下記の具体的なコードも紹介されている。  
```
import numpy as np
import cv2
import torch
import kornia.feature as KF

# Load model
device = torch.device('cuda')
matcher = KF.LoFTR(pretrained=None)
matcher.load_state_dict(torch.load("../input/kornia-loftr/loftr_outdoor.ckpt")['state_dict'])
matcher = matcher.to(device).eval()

# Inference
F_dict = {}
for i, sample_id, image0, image1 in enumerate(test_samples):    
    with torch.no_grad():
        correspondences = matcher(image0=image0, image1=image1)
    
    # 対応点
    mkpts0 = correspondences['keypoints0'].cpu().numpy()
    mkpts1 = correspondences['keypoints1'].cpu().numpy()
    
    # Calc fundamental matrix
    if len(mkpts0) > 7:
        F, _ = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.25, 0.9999, 100000)
        F_dict[sample_id] = F
    else:
        F_dict[sample_id] = np.zeros((3, 3))
```

[3次元コンピュータビジョン計算ハンドブック](https://www.amazon.co.jp/3%E6%AC%A1%E5%85%83%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%82%BF%E3%83%93%E3%82%B8%E3%83%A7%E3%83%B3%E8%A8%88%E7%AE%97%E3%83%8F%E3%83%B3%E3%83%89%E3%83%96%E3%83%83%E3%82%AF-%E9%87%91%E8%B0%B7-%E5%81%A5%E4%B8%80/dp/4627817916)  
上の本には、下記の方法が記載されている。(p.75, 80~84)  
1. 基礎行列から内部パラメータと外部パラメータを推定する方法 
2. 内部パラメータと外部パラメータからカメラ行列を計算する方法
3. カメラ行列を用いて２画像間の対応点から３次元座標を計算する方法
* 1, 2のように基礎行列からカメラ行列を直接推定することを自己校正という
* 3の方法は三角測量という